# ATN: Augmented Transition Network
* wikipedia: https://en.wikipedia.org/wiki/Augmented_transition_network
* geeksforgeeks: https://www.geeksforgeeks.org/augmented-transition-networks-in-natural-language-processing/

* Transition network grammars for natural language analysis: https://dl.acm.org/doi/10.1145/355598.362773

# Components
* states
* transitions
* registers
* tests
* actions

In [1]:
# Define the ATN structure
class ATN:
    def __init__(self):
        self.states = {}
        self.registers = {}

    def add_state(self, state_name):
        """添加状态"""
        self.states[state_name] = State(state_name)

    def add_transition(self, from_state, to_state, condition, action):
        """添加转移"""
        if from_state in self.states and to_state in self.states:
            self.states[from_state].add_transition(to_state, condition, action)
        else:
            raise ValueError("State not found")

    def set_register(self, register_name, value):
        """设置寄存器值"""
        self.registers[register_name] = value

    def get_register(self, register_name):
        """获取寄存器值"""
        return self.registers.get(register_name, None)

    def parse(self, input_tokens):
        """解析输入"""
        # 当前状态: 开始状态
        current_state = self.states["START"]
        # 遍历输入
        for token in input_tokens:
            # 获取下一状态
            next_state, action = current_state.get_next_state(
                token, self.registers)
            if next_state:
                # 执行动作
                action(self.registers)
                # 更新当前状态
                current_state = self.states[next_state]
            else:
                raise ValueError("Parsing Error")
        # 返回状态
        return current_state

# Define the State class


class State:
    def __init__(self, name):
        self.name = name
        self.transitions = []

    def add_transition(self, to_state, condition, action):
        """添加转移"""
        # 转移的数据结构: (to状态, 条件, 动作)
        self.transitions.append((to_state, condition, action))

    def get_next_state(self, token, registers):
        """获取下一状态"""
        # 遍历所有转移
        for (to_state, condition, action) in self.transitions:
            # 满足条件的记号, 返回: (to状态, 动作)
            if condition(token, registers):
                return to_state, action
        return None, None

# Define conditions and actions

# 条件: 是否是冠词
def condition_is_article(token, registers):
    return token.lower() in ["the", "a", "an"]

# 条件: 是否是名词
def condition_is_noun(token, registers):
    return token.lower() in ["dog", "cat", "apple"]

# 条件: 是否是动词
def condition_is_verb(token, registers):
    return token.lower() in ["runs", "jumps", "eats"]

# 动作: 设置冠词
def action_set_article(registers):
    registers["article"] = True

# 动作: 设置名词
def action_set_noun(registers):
    registers["noun"] = True

# 动作: 设置动词
def action_set_verb(registers):
    registers["verb"] = True


# Example usage
# ATN.drawio
atn = ATN()
atn.add_state("START")
atn.add_state("ARTICLE")
atn.add_state("NOUN")
atn.add_state("VERB")

atn.add_transition("START", "ARTICLE",
condition_is_article, action_set_article)
atn.add_transition("ARTICLE", "NOUN", condition_is_noun, action_set_noun)
atn.add_transition("NOUN", "VERB", condition_is_verb, action_set_verb)

input_sentence = ["The", "dog", "runs"]
final_state = atn.parse(input_sentence)

print("Final state:", final_state.name)
print("Registers:", atn.registers)


Final state: VERB
Registers: {'article': True, 'noun': True, 'verb': True}
